In [1]:
from src.ingest_redis import *
from src.ingest_chroma import *
from src.ingest_milvus import * 
from src.search import *
import time
import pandas as pd

In [2]:
# set up dataframe to track experiment results
cols =  ['database', 'chunk_size', 'overlap', 'clean', 'embedding', 'chunks_processed', 'time_to_process', 'used_memory_mb', 'query_time']
results = pd.DataFrame(columns = cols)

In [3]:
with open('example_queries.txt', 'r') as file:

    # Skip lines that don't contain actual queries (headers, empty lines) and extract example queries
    queries = [line.strip() for line in file if line.strip() and not line.strip().startswith('#') and not line.strip().startswith('##')]
    queries = [q.split('. ', 1)[1] if '. ' in q else q for q in queries]
    
    # Print total count
    print(f"Total queries: {len(queries)}")

Total queries: 500


In [5]:
# test different chunk/overlap/clean combos for filling redis database
embedding_model = 'nomic-embed-text'
db = 'redis'

# loop through different options
for chunk in [100,300,500,1000]:
    for overlap in [0, 50, 100]:

        if overlap >= chunk:
            continue
        for clean in [True, False]:

            # clear store before starting
            clear_redis_store()

            # create and fill redis store
            start = time.time()
            create_hnsw_index()
            chunk_count = process_pdfs_redis("data/", chunk_size=chunk, overlap=overlap, clean = clean)
            to_fill = time.time() - start
            print(f'Index with chunk size {chunk} and overlap {overlap} created in {round(to_fill, 2)} seconds')


            # Get memory usage info
            memory_info = redis_client.info('memory')
            used_memory = memory_info['used_memory'] 
            used_memory_mb = used_memory / (1024 * 1024)
            print(f"It uses {used_memory_mb} MB")

            # test retrieval speed
            start = time.time()
            for query in queries:
                query_redis(query)
            to_search = time.time()- start
            print(f'Search with chunk size {chunk} and overlap {overlap} completed in {round(to_search, 2)} seconds')

            # add results to result dataframe
            new_row = [db, chunk, overlap, clean, embedding_model, chunk_count, to_fill, used_memory_mb, to_search]
            results = pd.concat([results, pd.DataFrame([new_row], columns=cols)], axis = 0)

Index with chunk size 100 and overlap 0 created in 9.11 seconds
Search with chunk size 100 and overlap 0 completed in 9.84 seconds


/var/folders/b1/t_5vbv0n4_j3m_7gy69b59tm0000gn/T/ipykernel_52017/933467114.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([new_row], columns=cols)], axis = 0)


Index with chunk size 100 and overlap 0 created in 11.61 seconds
Search with chunk size 100 and overlap 0 completed in 9.71 seconds
Index with chunk size 100 and overlap 50 created in 17.04 seconds
Search with chunk size 100 and overlap 50 completed in 11.14 seconds
Index with chunk size 100 and overlap 50 created in 18.03 seconds
Search with chunk size 100 and overlap 50 completed in 10.03 seconds
Index with chunk size 300 and overlap 0 created in 8.95 seconds
Search with chunk size 300 and overlap 0 completed in 9.38 seconds
Index with chunk size 300 and overlap 0 created in 10.15 seconds
Search with chunk size 300 and overlap 0 completed in 9.4 seconds
Index with chunk size 300 and overlap 50 created in 9.33 seconds
Search with chunk size 300 and overlap 50 completed in 9.08 seconds
Index with chunk size 300 and overlap 50 created in 10.15 seconds
Search with chunk size 300 and overlap 50 completed in 9.23 seconds
Index with chunk size 300 and overlap 100 created in 8.61 seconds
Sea

In [6]:
# test different chunk/overlap/clean combos for filling redis database
embedding_model = 'nomic-embed-text'
embedding_size = 768
db = 'chroma'

# loop through different options
for chunk in [100,300,500,1000]:
    for overlap in [0, 50, 100]:
        if overlap >= chunk:
            continue
        for clean in [True, False]:

            # create and fill chroma store
            start = time.time()
            collection = create_chroma_index(embedding_model)
            collection, chunk_count = process_pdfs_chroma(collection, "data/", chunk_size=chunk, overlap=overlap, clean = clean)
            to_fill = time.time() - start
            print(f'Index with chunk size {chunk} and overlap {overlap} created in {round(to_fill, 2)} seconds')

            num_vectors = len(collection.get()["ids"])  # Number of stored items
            embedding_size = 768  # Adjust based on your embedding model
            float_size = np.dtype(np.float32).itemsize  # 4 bytes per float

            memory_usage_bytes = num_vectors * embedding_size * float_size
            memory_usage_mb = memory_usage_bytes / (1024 * 1024)
            print(f"It uses {memory_usage_bytes} MB")

            # test searching speed
            start = time.time()
            query_chroma(collection, queries)
            to_search = time.time()-start
            print(f'Search with chunk size {chunk} and overlap {overlap} completed in {round(to_search, 2)} seconds')

            # add results to result dataframe
            new_row = [db, chunk, overlap, clean, embedding_model, chunk_count, to_fill, memory_usage_mb, to_search]
            results = pd.concat([results, pd.DataFrame([new_row], columns=cols)], axis = 0)

Index with chunk size 100 and overlap 0 created in 14.7 seconds
It uses 1274880 MB
Search with chunk size 100 and overlap 0 completed in 8.57 seconds
Index with chunk size 100 and overlap 0 created in 15.95 seconds
It uses 1299456 MB
Search with chunk size 100 and overlap 0 completed in 8.75 seconds
Index with chunk size 100 and overlap 50 created in 21.35 seconds
It uses 1956864 MB
Search with chunk size 100 and overlap 50 completed in 8.34 seconds
Index with chunk size 100 and overlap 50 created in 22.63 seconds
It uses 2055168 MB
Search with chunk size 100 and overlap 50 completed in 8.27 seconds
Index with chunk size 300 and overlap 0 created in 14.5 seconds
It uses 930816 MB
Search with chunk size 300 and overlap 0 completed in 8.59 seconds
Index with chunk size 300 and overlap 0 created in 14.35 seconds
It uses 936960 MB
Search with chunk size 300 and overlap 0 completed in 7.41 seconds
Index with chunk size 300 and overlap 50 created in 12.63 seconds
It uses 967680 MB
Search wit

In [4]:
# test different chunk/overlap/clean combos for filling redis database
embedding_model = 'nomic-embed-text'
embedding_size = 768
db = 'milvus'

# loop through different options
for chunk in [100,300,500,1000]:
    for overlap in [0, 50, 100]:
        if overlap >= chunk:
            continue
        for clean in [True, False]:

            # create and fill Milvus collection
            start = time.time()
            collection = create_milvus_collection(embed_dim=embedding_size)
            collection, num_vectors = process_pdfs_milvus(collection, "data/", chunk_size=chunk, overlap=overlap, clean=clean)
            index_time = time.time() - start
            print(f"Index with chunk size {chunk} and overlap {overlap} created in {round(index_time, 2)} seconds")

            # get memory usage
            float_size = np.dtype(np.float32).itemsize  # 4 bytes per float
            memory_usage_bytes = num_vectors * embedding_size * float_size
            memory_usage_mb = memory_usage_bytes / (1024 * 1024)
            print(f"It uses {round(memory_usage_mb, 2)} MB")

            # test searcging speed
            start = time.time()
            for query in queries:
                query_milvus(collection, query, top_k=5)
            search_time = time.time() - start
            print(f"Search with chunk size {chunk} and overlap {overlap} completed in {round(search_time, 2)} seconds")

            # add results to resul dataframe
            new_row = [db, chunk, overlap, clean, embedding_model, num_vectors, index_time, memory_usage_mb, search_time]
            results = pd.concat([results, pd.DataFrame([new_row], columns=cols)], ignore_index=True)


Index with chunk size 100 and overlap 0 created in 12.19 seconds
It uses 1.22 MB
Search with chunk size 100 and overlap 0 completed in 17.14 seconds


/var/folders/b1/t_5vbv0n4_j3m_7gy69b59tm0000gn/T/ipykernel_55250/3922220688.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([new_row], columns=cols)], ignore_index=True)


Index with chunk size 100 and overlap 0 created in 13.81 seconds
It uses 1.24 MB
Search with chunk size 100 and overlap 0 completed in 17.19 seconds
Index with chunk size 100 and overlap 50 created in 15.13 seconds
It uses 1.87 MB
Search with chunk size 100 and overlap 50 completed in 17.26 seconds
Index with chunk size 100 and overlap 50 created in 17.39 seconds
It uses 1.96 MB
Search with chunk size 100 and overlap 50 completed in 16.32 seconds
Index with chunk size 300 and overlap 0 created in 10.61 seconds
It uses 0.89 MB
Search with chunk size 300 and overlap 0 completed in 16.88 seconds
Index with chunk size 300 and overlap 0 created in 12.24 seconds
It uses 0.89 MB
Search with chunk size 300 and overlap 0 completed in 15.84 seconds
Index with chunk size 300 and overlap 50 created in 11.04 seconds
It uses 0.92 MB
Search with chunk size 300 and overlap 50 completed in 17.13 seconds
Index with chunk size 300 and overlap 50 created in 11.34 seconds
It uses 0.93 MB
Search with chunk 

In [ ]:
results[results['database']=='chroma'].head()

In [5]:
results[results['database']=='milvus'].head()

,database,chunk_size,overlap,clean,embedding,chunks_processed,time_to_process,used_memory_mb,query_time
0,milvus,100,0,True,nomic-embed-text,415,12.192802,1.215820,17.138979
1,milvus,100,0,False,nomic-embed-text,423,13.813842,1.239258,17.191782
2,milvus,100,50,True,nomic-embed-text,637,15.134689,1.866211,17.259680
3,milvus,100,50,False,nomic-embed-text,669,17.386459,1.959961,16.320756
4,milvus,300,0,True,nomic-embed-text,303,10.613719,0.887695,16.875774


In [ ]:
results.reset_index().loc[35, 'chunks_processed'][1]

In [ ]:
results.to_csv('experiment_results.csv')